Comapre the time range of both the old data and new data

In [1]:
# libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# old data

station_id=5001
file_path = os.path.join(r'D:\FMF magisterij\Matematika z računalnikom', f'ELES-MAS-{station_id}-2020-2023.csv.gz')
data = pd.read_csv(file_path, compression='gzip')
data = data[data['data_validity'] <= 32]
data.drop('measurement_location_inst_id', axis=1, inplace=True)
data['acquisition_timestamp'] = pd.to_datetime(data['acquisition_timestamp'], utc=True)


# rearrange the data
old_data = data.pivot_table(index='acquisition_timestamp', columns='parameter_type', values='value').reset_index()
old_data.set_index('acquisition_timestamp', inplace=True)



# fill NaN values in the row with the values of next row with non NaN values
old_data = old_data.fillna(method='bfill')


print('Shape of data', old_data.shape)

old_data.head()

Shape of data (271508, 7)


parameter_type,AIR_PRESSURE,AMBIENT_TEMPERATURE,RAIN_INTENSITY,RELATIVE_HUMIDITY,SOLAR_RADIATION_INTENSITY,WIND_DIRECTION,WIND_SPEED
acquisition_timestamp,,,,,,,
2020-10-13 10:20:00+00:00,952.8,7.4,0.0,63.9,285.6,294.0,2.45
2020-10-13 10:25:00+00:00,952.8,7.5,0.0,62.5,262.0,294.0,2.45
2020-10-13 10:30:00+00:00,952.8,7.4,0.0,61.9,270.9,282.0,2.81
2020-10-13 10:35:00+00:00,952.8,7.5,0.0,62.2,307.4,301.0,3.28
2020-10-13 10:40:00+00:00,952.8,7.6,0.0,62.2,342.0,289.0,1.27


In [4]:
# new data

file_path = os.path.join(r'D:\FMF magisterij\Matematika z računalnikom', f'test_weather_data.csv')
new_data = pd.read_csv(file_path)

# drop the 'index' column
new_data = new_data.drop(columns=['index'])

new_data.head()

,time,airT,rainIntensity,rH,solarRadiance,windDir,windSpeed,pressure
0,2020-11-05 12:15:00+00:00,10.9,0.0,76.4,111.4,233.0,1.13,99010.0
1,2020-11-05 12:20:00+00:00,10.9,0.0,76.7,115.5,245.0,1.17,99010.0
2,2020-11-05 12:25:00+00:00,11.0,0.0,75.5,117.2,243.0,1.64,99010.0
3,2020-11-05 12:30:00+00:00,11.0,0.0,76.0,118.8,226.0,1.54,99000.0
4,2020-11-05 12:35:00+00:00,11.0,0.0,74.2,120.4,221.0,1.90,99000.0


In [15]:
# rename columns to match the old format
new_data = new_data.rename(columns={
    'time': 'acquisition_timestamp',
    'airT': 'AMBIENT_TEMPERATURE',
    'rainIntensity': 'RAIN_INTENSITY',
    'rH': 'RELATIVE_HUMIDITY',
    'solarRadiance': 'SOLAR_RADIATION_INTENSITY',
    'windDir': 'WIND_DIRECTION',
    'windSpeed': 'WIND_SPEED',
    'pressure': 'AIR_PRESSURE'
})

# convert acquisition_timestamp to datetime
new_data['acquisition_timestamp'] = pd.to_datetime(new_data['acquisition_timestamp'], utc=True)




# set the 'acquisition_timestamp' as the index
new_data = new_data.set_index('acquisition_timestamp')

# rearrange columns to match the old format
new_data = new_data[[
    'AIR_PRESSURE', 'AMBIENT_TEMPERATURE', 'RAIN_INTENSITY', 
    'RELATIVE_HUMIDITY', 'SOLAR_RADIATION_INTENSITY', 
    'WIND_DIRECTION', 'WIND_SPEED'
]]

new_data.head()


,AIR_PRESSURE,AMBIENT_TEMPERATURE,RAIN_INTENSITY,RELATIVE_HUMIDITY,SOLAR_RADIATION_INTENSITY,WIND_DIRECTION,WIND_SPEED
acquisition_timestamp,,,,,,,
2020-11-05 12:15:00+00:00,99010.0,10.9,0.0,76.4,111.4,233.0,1.13
2020-11-05 12:20:00+00:00,99010.0,10.9,0.0,76.7,115.5,245.0,1.17
2020-11-05 12:25:00+00:00,99010.0,11.0,0.0,75.5,117.2,243.0,1.64
2020-11-05 12:30:00+00:00,99000.0,11.0,0.0,76.0,118.8,226.0,1.54
2020-11-05 12:35:00+00:00,99000.0,11.0,0.0,74.2,120.4,221.0,1.90


In [16]:
# compare the timerange

# extract time range for both datasets
old_start = old_data.index.min()
old_end = old_data.index.max()
new_start = new_data.index.min()
new_end = new_data.index.max()

# check if the time ranges overlap
def time_ranges_overlap(start1, end1, start2, end2):
    return max(start1, start2) <= min(end1, end2)

# display time ranges and check for overlap
print(f"Old data time range: {old_start} to {old_end}")
print(f"New data time range: {new_start} to {new_end}")
print("Do the time ranges overlap?", time_ranges_overlap(old_start, old_end, new_start, new_end))

Old data time range: 2020-10-13 10:20:00+00:00 to 2023-05-31 21:55:00+00:00
New data time range: 2020-11-05 12:15:00+00:00 to 2023-05-31 21:55:00+00:00
Do the time ranges overlap? True


In [18]:
# extract the overlapping data and store it to the new variable

# find overlapping time range
overlap_start = max(old_start, new_start)
overlap_end = min(old_end, new_end)

# extract overlapping data
overlapping_old_data = old_data.loc[overlap_start:overlap_end]
overlapping_new_data = new_data.loc[overlap_start:overlap_end]

overlapping_data = pd.concat([overlapping_old_data, overlapping_new_data])

# display the overlapping data
print(overlapping_data)

# save the overlapping data to a new CSV file
overlapping_data.to_csv('overlapping_data.csv')

                           AIR_PRESSURE  AMBIENT_TEMPERATURE  RAIN_INTENSITY  \
acquisition_timestamp                                                          
2020-11-05 12:15:00+00:00         976.7                 11.0             0.0   
2020-11-05 12:20:00+00:00         976.7                 11.0             0.0   
2020-11-05 12:25:00+00:00         976.7                 11.0             0.0   
2020-11-05 12:30:00+00:00         976.7                 11.0             0.0   
2020-11-05 12:35:00+00:00         976.7                 11.0             0.0   
...                                 ...                  ...             ...   
2023-05-31 21:35:00+00:00       97710.0                 15.1             0.0   
2023-05-31 21:40:00+00:00       97700.0                 15.0             0.0   
2023-05-31 21:45:00+00:00       97710.0                 15.1             0.0   
2023-05-31 21:50:00+00:00       97700.0                 15.0             0.0   
2023-05-31 21:55:00+00:00       97700.0 